### Packages  

First you need to install all packages which are compatible with the latset version of python. 

Installed packages need to be upgraded.

In [1]:
!pip3 install -r requirements --user

     |████████████████████████████████| 14.9 MB 15.1 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
phased-lstm-keras 1.0.2 requires keras>=2.0.2, but you have keras 0.3.1 which is incompatible.
keras-vgg-buddy 0.0.5 requires Keras>=0.3.2, but you have keras 0.3.1 which is incompatible.
keras-rtst 0.0.9 requires Keras==0.3.3, but you have keras 0.3.1 which is incompatible.
keras-rl 0.4.2 requires keras>=2.0.7, but you have keras 0.3.1 which is incompatible.
keras-resnet 0.2.0 requires keras>=2.2.4, but you have keras 0.3.1 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from multiprocessing import Pool
import tensorflow as tf
from tensorflow.python.ops import nn


py_file_location = "..."
os.path.abspath(os.path.join(os.path.dirname(py_file_location), os.path.pardir))

from model_DL import *
from functions import *
from metrics import *
from DNN_metrics import *
from data_generator import *
from loss_function import *
from optimization import *



#tf.device('GPU:1') 

2022-03-10 14:48:44.451415: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Choose GPU if it is available 

In [2]:
print(tf.config.list_physical_devices())
tf.device('GPU:1') 

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


2022-03-10 14:48:45.782722: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-10 14:48:45.784170: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-10 14:48:45.851967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-03-10 14:48:45.852461: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:5e:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-03-10 14:48:45.852507: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-10 14:4

calhost/replica:0/task:0/device:GPU:0 with 3943 MB memory) -> physical GPU (device: 0, name: Quadro RTX 8000, pci bus id: 0000:3b:00.0, compute capability: 7.5)
2022-03-10 14:48:46.867518: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:1 with 42242 MB memory) -> physical GPU (device: 1, name: Quadro RTX 8000, pci bus id: 0000:5e:00.0, compute capability: 7.5)


### Channel gain creation

First, you can choose the type of channel to perform the simulations. To do this, simply select the type by entering the number of the desired channel type 

In [3]:
h_PP, h_PR, h_RP, h_SS, h_SR, h_RS, h_SP, h_PS = channel_type() # train data-type


        1. Channel gain with gaussian fading [1]
        2. Channel gain with Anne model [2]
        3. Channel gain with Uniform distribution[3] 
        4. Channel gain with Rician fading [4]
        5. Channel gain with Nakagami fading [5]
        6.Exit/Quit
        
Select channel gain
1
Channel gain created


In [3]:
t_h_PP, t_h_PR, t_h_RP, t_h_SS, t_h_SR, t_h_RS, t_h_SP, t_h_PS = channel_type() # test data-type


        1. Channel gain with gaussian fading [1]
        2. Channel gain with Anne model [2]
        3. Channel gain with Uniform distribution[3] 
        4. Channel gain with Rician fading [4]
        5. Channel gain with Nakagami fading [5]
        6.Exit/Quit
        
Select channel gain
1
Channel gain created


# Convert channel coefficient to channel gain

In [5]:
g_PP, g_PR, g_RP, g_SS, g_SR, g_RS, g_SP, g_PS =\
np.power(h_PP, 2), np.power(h_PR, 2), np.power(h_RP, 2), np.power(h_SS, 2)\
, np.power(h_SR, 2), np.power(h_RS, 2), np.power(h_SP, 2), np.power(h_PS, 2)

In [4]:
t_g_PP, t_g_PR, t_g_RP, t_g_SS, t_g_SR, t_g_RS, t_g_SP, t_g_PS =\
np.power(t_h_PP, 2), np.power(t_h_PR, 2), np.power(t_h_RP, 2), np.power(t_h_SS, 2)\
, np.power(t_h_SR, 2), np.power(t_h_RS, 2), np.power(t_h_SP, 2), np.power(t_h_PS, 2)

### Filter Data

After data creation, it must be filtred to avoide the division by zero on the custom loss function for that we maintint only channal gain greater then a fixed threshold s.


In [5]:
g_RP, g_PP, g_SR, g_PR, g_SS, g_RS, g_SP, g_PS = data_filter(g_RP, g_PP, g_SR, g_PR, g_SS, g_RS, g_SP, g_PS)

t_g_RP, t_g_PP, t_g_SR, t_g_PR, t_g_SS, t_g_RS, t_g_SP, t_g_PS = data_filter(t_g_RP, t_g_PP, t_g_SR, t_g_PR, t_g_SS, t_g_RS, t_g_SP, t_g_PS)

### Dataset generation
Create the dataset and save it on a specific path

1 - data size must be fixed

In [8]:
Nbr_train = int(1E6)

Nbr_test = int(2E5) 

In [9]:
# # choose your directory path  
project_sub_path = "Dataset-GIT"
  
# Parent Directory path
parent_dir = ""
  
# Path
try : 

    path = os.path.join(parent_dir, project_sub_path)
    os.mkdir(path)
    print("Directory '% s' created" % project_sub_path)
except FileExistsError : 
    pass

In [10]:
np.savez(os.path.join(parent_dir,project_sub_path,'dataset_train.npz'),
         g_PP=g_PP,
         g_PS=g_PS,
         g_PR=g_PR,
         g_SP=g_SP,
         g_SS=g_SS,
         g_SR=g_SR,
         g_RP=g_RP,
         g_RS=g_RS
         ) 

print("\n training set generation finished")


 training set generation finished


### Bruteforce
The bruteforce method is applied only for the test set to use it as a benchmark with the DNN predicted results

In [ ]:
bf_results = benchmark_generator(np.stack([t_g_RP[:Nbr_test], t_g_PP[:Nbr_test], t_g_SR[:Nbr_test], t_g_PR[:Nbr_test], t_g_SS[:Nbr_test], t_g_RS[:Nbr_test], t_g_SP[:Nbr_test], t_g_PS[:Nbr_test]], axis=1))

np.savez(os.path.join(parent_dir,project_sub_path,'dataset_test_3.npz'),
         g_PP=t_g_PP[:Nbr_test],
         g_PS=t_g_PS[:Nbr_test],
         g_PR=t_g_PR[:Nbr_test],
         g_SP=t_g_SP[:Nbr_test],
         g_SS=t_g_SS[:Nbr_test],
         g_SR=t_g_SR[:Nbr_test],
         g_RP=t_g_RP[:Nbr_test],
         g_RS=t_g_RS[:Nbr_test],
         R_S=bf_results[:,8], 
         Alpha=bf_results[:,9], 
         p_R=bf_results[:,10], 
         p_S=bf_results[:,11]
         ) 

print("\n test set generation finished")

### Split dataset
load the dataset if it's already created

In [7]:
#Nbr_train = int(1E6)

#Nbr_test = int(2E5) 

# # choose your directory path  
project_sub_path = "Dataset-GIT"
  
# Parent Directory path
parent_dir = ""


### Train ###

dataset_train = np.load(os.path.join(parent_dir,project_sub_path,'dataset_train.npz'))

g_11_tr = dataset_train['g_PP'][:Nbr_train]
g_12_tr = dataset_train['g_PS'][:Nbr_train]
g_1R_tr = dataset_train['g_PR'][:Nbr_train]
g_21_tr = dataset_train['g_SP'][:Nbr_train]
g_22_tr = dataset_train['g_SS'][:Nbr_train]
g_2R_tr = dataset_train['g_SR'][:Nbr_train]
g_R1_tr = dataset_train['g_RP'][:Nbr_train]
g_R2_tr = dataset_train['g_RS'][:Nbr_train]

x_train = np.stack([g_R1_tr, g_11_tr, g_2R_tr, g_1R_tr, g_22_tr, g_R2_tr, g_21_tr, g_12_tr], axis=1)


### Test ### 

dataset_test = np.load(os.path.join(parent_dir,project_sub_path,'dataset_test.npz'))


g_11_ts = dataset_test['g_PP'][:Nbr_test]
g_12_ts = dataset_test['g_PS'][:Nbr_test]
g_1R_ts = dataset_test['g_PR'][:Nbr_test]
g_21_ts = dataset_test['g_SP'][:Nbr_test]
g_22_ts = dataset_test['g_SS'][:Nbr_test]
g_2R_ts = dataset_test['g_SR'][:Nbr_test]
g_R1_ts = dataset_test['g_RP'][:Nbr_test]
g_R2_ts = dataset_test['g_RS'][:Nbr_test]

R_S_ts = dataset_test['R_S']
Alpha_ts = dataset_test['Alpha']
P_R_ts = dataset_test['p_R']
P_S_ts = dataset_test['p_S']


x_test = np.stack([g_R1_ts, g_11_ts, g_2R_ts, g_1R_ts, g_22_ts, g_R2_ts, g_21_ts, g_12_ts], axis=1)

y_test = np.stack([R_S_ts, Alpha_ts, P_R_ts, P_S_ts], axis=1)


## Training phase 

### train DNN with $\lambda \in [10^{-1},...,10^{2}]$

In [ ]:
tau = 0.25

metrics = [Achievable_Rate(tau), QoS_Violation(tau), Primary_Achievable_Rate_Degradation, Primary_ARD_Percentage(tau)] 

Epochs = 1000 # Epochs number

BS = 4096 # Batch_size

VS = 0.2 # Validation set

LD = {'10_-1':10**-1, '10_-0.75':10**-0.75, '10_-0.5':10**-0.5, '10_-0.25':10**-0.25, '10_0':10**0, '10_0.25':10**0.25, '10_0.5':10**0.5, '10_0.75':10**0.75, '10_1':10**1, '10_1.25':10**1.25, '10_1.5':10**1.5, '10_1.75':10**1.75, '10_2':10**2}

LR = 10**-4 #{'10_-4':10**-4}

root_dir ='DNN'

for ld_k, ld_v in LD.items():
    
    #Create a new directory (a folder) in Drive

    lambda_dir = root_dir+'/lambda = '+ld_k+'/weights/'
  
    history_dir = root_dir+'/lambda = '+ld_k+'/history/'

    tf.io.gfile.makedirs(lambda_dir)

    tf.io.gfile.makedirs(history_dir)

    #for lr_k, lr_v in LR.items(): add a loop if browsing learning rate needed

    model = get_model_DF(x_train, loss_DF_WN(ld_v,tau), metrics,'sigmoid', custom_sigmoid, custom_sigmoid, LR) #lr_v
    history = model.fit(x_train, x_train, epochs=Epochs, batch_size=BS, validation_split = VS)

    model.save(lambda_dir+ld_k+'.h5')
    np.save(history_dir+ld_k+'.npy',history.history)

